In [1]:
import time
from zipfile import ZipFile
file_name = "test_data/2022-ntust-information-retrieval-hw2.zip"

with ZipFile(file_name, 'r',) as zip:
    zip.extractall(path = 'test_data/extractHere/2022-ntust-information-retrieval-hw2')
    print('Done!!')
    All_lst = zip.namelist()

start = time.time()
Query_lst = []
Doc_lst = []
for i in All_lst:
    if "documents/" in i:
        Doc_lst.append(i)
    elif "queries/" in i:
        Query_lst.append(i)

print("Query size: ", len(Query_lst),' / ',"Document size: ", len(Doc_lst))

end = time.time()
print("execution time(all small n): %f" %(end - start))

Done!!
Query size:  100  /  Document size:  5000
execution time(all small n): 0.000000


In [2]:
# 可以在這建立lexicon
lexicon = {}
data_addr = 'test_data/extractHere/2022-ntust-information-retrieval-hw2/'

start = time.time()
for i in Query_lst:
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        lexicon[term] = 0

avg_Doc_len = 0
# 要把document所有字也加進lexicon?
for i in Doc_lst:
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        avg_Doc_len = avg_Doc_len + 1
        lexicon[term] = 0

avg_Doc_len = avg_Doc_len/len(Doc_lst)

print("number of terms: ", len(lexicon))
print("avg_Doc_len: ", avg_Doc_len)

end = time.time()
print("execution time(all small n): %f" %(end - start))

number of terms:  262226
avg_Doc_len:  1098.733
execution time(all small n): 20.526171


In [3]:
import numpy as np

# 計算IDF中的n_i
# Q_small_n = lexicon.copy()
D_small_n = lexicon.copy()
check = lexicon.copy()

start = time.time()
# for q in Query_lst:
#     file = open(data_addr + q, errors = 'ignore').read().split()
#     for term in check:
#         check[term] = False
    
#     for term in file:
#         if check[term] == False:
#             Q_small_n[term] = Q_small_n[term] + 1 
#             check[term] = True

for d in Doc_lst:
    file = open(data_addr + d, errors = 'ignore').read().split()
    for term in check:
        check[term] = False
        
    for term in file:
        if term in lexicon and check[term] == False:
            D_small_n[term] = D_small_n[term] + 1
            check[term] = True

end = time.time()
print("execution time(all small n): %f" %(end - start))

# Q_small_n = list(Q_small_n.values())
D_small_n = list(D_small_n.values())
print("D_small_n: ", len(D_small_n))
avoid_zero = np.ones(len(lexicon))
print("avoid_zero: ", len(avoid_zero))

# print("Q_small_n: ", Q_small_n)
# print("D_small_n: ", D_small_n)

execution time(all small n): 156.788369
D_small_n:  262226
avoid_zero:  262226


In [4]:
import math


D_small_n_temp = D_small_n
D_small_n = np.ones(len(D_small_n))* (len(lexicon)+ 0.5) - D_small_n
D_small_n = np.divide(D_small_n, D_small_n_temp + np.ones(len(D_small_n))* 0.5)
D_small_n = np.log10(D_small_n + avoid_zero)
# D_small_n = (D_small_n + avoid_zero)

# for i in D_small_n:
#     i = math.log(i+1, 1.1)


#每一迴圈計算一個document相對於所有lexicon的tf-idf，並存成一個list
start = time.time()
D_tfidf_lst = []

for i in range(len(Doc_lst)):
    D_k = lexicon.copy()
    D_tfidf = [0 for _  in range(len(lexicon))]
    
    Doc_len = 0
    file = open(data_addr + Doc_lst[i], errors = 'ignore').read().split()
    for term in file:
        Doc_len = Doc_len + 1
        if term in lexicon:
            D_k[term] = D_k[term] + 1

# BM Match, K_1, b
    K_1 = 1.7
    S_1 = K_1 + 1
    
    b = 0.75
    delta = 5
    
    K_3 = 1000
    S_3 = K_3 + 1
    
    D_k = list(D_k.values())
#     D_k = np.log2(D_k + avoid_zero)
#     D_k = np.log10(D_k + avoid_zero)

    D_k = np.divide(D_k, (1 - b + b* (Doc_len/ avg_Doc_len)))

    D_k = D_k + np.ones(len(D_k))* delta
    D_tfidf = np.multiply(S_1, D_k)
    D_k = np.ones(len(D_k))* K_1 + D_k
    D_tfidf = np.divide(D_tfidf, D_k)
  
    D_tfidf_lst.append(D_tfidf)
end = time.time()
print("execution time(all D_tfidf): %f" %(end - start))

execution time(all D_tfidf): 181.342932


In [5]:
import csv
import winsound
import time

start = time.time()
# 每一迴圈輸出一個Query的結果到test.csv
with open("test_data/test.csv", "w", newline='') as Answer_csv:
    Answer_csv.truncate()
    writer = csv.writer(Answer_csv)
    writer.writerow(['Query', 'RetrievedDocuments'])
    
    for ans in range(len(Query_lst)):
        Q_k = lexicon.copy()
        Q_tfidf = [0 for _ in range(len(lexicon))]
        sim_list = [0 for _ in range(len(Doc_lst))]

        file = open(data_addr + Query_lst[ans], errors = 'ignore').read().split()
        for term in file:
            if term in lexicon:
                Q_k[term] = Q_k[term] + 1

                
        Q_k = list(Q_k.values())
        
# BM Match, Query weighting.    
#         Q_k = np.log10(Q_k + avoid_zero)
#         for i in Q_k:
#             i = math.log(i+1, 100)
        
        Q_tfidf = np.multiply(S_3, Q_k)
        Q_k = np.ones(len(Q_k))* K_3 + Q_k
        Q_tfidf = np.divide(Q_tfidf, Q_k)
        
        for i in range(len(D_tfidf_lst)):
            sim_temp = np.multiply(Q_tfidf, D_tfidf_lst[i])
            sim_temp = np.multiply(sim_temp, D_small_n)
            sim_list[i] = np.sum(sim_temp)
    
        sorted_Doc = sorted(range(len(sim_list)), key=lambda k: sim_list[k], reverse = True)

        test_str = Doc_lst[sorted_Doc[0]].replace('documents/', '').replace(".txt", '')
        for i in range(1, len(Doc_lst)): 
            test_str = test_str + ' ' + Doc_lst[sorted_Doc[i]].replace('documents/', '').replace(".txt", '')
    
        writer.writerow([Query_lst[ans].replace('queries/', '').replace(".txt", '')] + [test_str])

end = time.time()
print("execution time(all Q_tfidf): %f" %(end - start))

duration = 500  # millisecond
freq = 440  # Hz
for i in range(5):
    winsound.Beep(freq, duration)

execution time(all Q_tfidf): 697.298992
